# Practice Exercises for Single Table SQL

Given a dataset and a set of questions, form the queries to answer the questions. All questions in this particular exercise only require a query to a single table. 

⚠️ *Note:* Please display minimal number of attributes (columns) needed to make sense of your answer. Not too many, not too few.


## Setup
The SQLite database for these exercises has been setup for you. For the curious, you can look at this file for the DDL commands to setup the tables and insert data.  `db_illum_ddl_5_3.sql`

Here is the schema for the database. The arrows show foreign keys, which is just fyi for now. 

![worker_project schema](worker_project_schema.png) 

In [1]:
# Make sure to always run this cell when you open this notebook
# Start the Jupyter SQL engine, connecting to a SQLite database 
%reload_ext sql 
%sql sqlite:///worker_project.db

0. In the command line, inspect the sqlite database (worker_project.db) to see what the tables and columns actually are. Any differences you observe? (Also, copy past your output in the cell below)

Yes, the columns don't have names


sqlite> SELECT * FROM Assign;
1001|101|30|
1001|103|20|5
1005|103|20|
1001|105|30|
1001|115|20|4
1019|110|20|5
1019|115|10|4
1025|110|10|
1030|110|10|

sqlite> SELECT * FROM Dept;
Accounting|101
Research|110

sqlite> SELECT * FROM Project;
1001|Jupiter|101|300000|01-Feb-2021|50
1005|Saturn|101|400000|01-Jun-2022|35
1019|Mercury|110|350000|15-Feb-2022|40
1025|Neptune|110|600000|01-Feb-2021|45
1030|Pluto|110|380000|15-Sep-2023|50

sqlite> SELECT * FROM Worker;
101|Smith|Tom|Accounting|01-Feb-1970|06-Jun-1993 |50000
103|Jones|Mary|Accounting|15-Jun-1975|20-Sep-2005|48000
105|Burns|Jane|Accounting|21-Sep-1980|12-Jun-2015|39000
110|Burns|Michael|Research|05-Apr-1977|10-Sep-2020|70000
115|Chin|Amanda|Research|22-Sep-1980|19-Jun-2022|60000





1. Get the names of all workers in the Accounting department.

In [2]:
%%sql
SELECT lastName, firstName FROM Worker WHERE deptName == "Accounting"

 * sqlite:///worker_project.db
Done.


lastName,firstName
Smith,Tom
Jones,Mary
Burns,Jane


2. Get the name of the employee in the Research department who has the lowest salary.

In [19]:
%%sql
 SELECT * FROM (SELECT lastName, firstName, salary FROM Worker) ORDER BY salary LIMIT 1;

 * sqlite:///worker_project.db
Done.


lastName,firstName,salary
Burns,Jane,39000


3. Get the details of the project with the highest budget.

In [23]:
%%sql
SELECT * FROM Project ORDER BY budget DESC LIMIT 1;

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1025,Neptune,110,600000,01-Feb-2021,45


## The following questions may require more than one query to answer. 
Show your work! Add as many SQL cells as needed for each question. Use comments to indicate how the output of each query is used to arrive at your answer.  

4. List out alphabetically the names of all workers working on Project 1001.

In [113]:
%%sql
-- First join the two tables
-- Order by last Name specify by ASC

SELECT lastName, firstName FROM Project CROSS JOIN Worker WHERE projNo = 1001 ORDER BY lastName ASC

 * sqlite:///worker_project.db
Done.


lastName,firstName
Burns,Jane
Burns,Michael
Chin,Amanda
Jones,Mary
Smith,Tom


5. Get an alphabetical list of names and corresponding ratings for all workers on any project managed by Michael Burns.

In [32]:
%%sql
SELECT firstName, lastName, rating FROM (SELECT empId, rating
FROM Assign 
WHERE projNo IN (SELECT projNo FROM Project WHERE projMgrId IN (SELECT empId FROM Worker WHERE lastName = 'Burns' AND firstName = 'Michael'))) as t
INNER JOIN Worker ON Worker.empId == t.empId;

 * sqlite:///worker_project.db
Done.


firstName,lastName,rating
Michael,Burns,5
Amanda,Chin,4
Michael,Burns,None
Michael,Burns,None


6. For all projects starting after Feb 1, 2022, find the project number and names of all workers assigned to them. 

In [50]:
%%sql
SELECT lastName, firstName, projNo FROM
(SELECT * FROM Assign WHERE projNo IN (SELECT projNo 
  FROM project
  WHERE startDate > '01-Feb-2022')) as t
  INNER JOIN Worker ON Worker.empId == t.empId;

 * sqlite:///worker_project.db
Done.


lastName,firstName,projNo
Jones,Mary,1005
Burns,Michael,1019
Chin,Amanda,1019
Burns,Michael,1030


7. Find the names of employees who are not assigned as managers any project.

In [53]:
%%sql
SELECT lastName, firstName FROM WORKER WHERE empId not in (SELECT projMgrId FROM Project)

 * sqlite:///worker_project.db
Done.


lastName,firstName
Jones,Mary
Burns,Jane
Chin,Amanda


8. Find details of any project with the string "urn" in the project name. 

In [85]:
%%sql
SELECT * FROM Project WHERE projName like ('%urn')

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1005,Saturn,101,400000,01-Jun-2022,35


9. List out the projects from most to least efficient. (hint: the project with the lowest budget is not necessarily the one that uses time and human resources effectively.)

In [100]:
%%sql
SELECT * FROM Project Order by budget/expectedDurationWeeks ASC

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1001,Jupiter,101,300000,01-Feb-2021,50
1030,Pluto,110,380000,15-Sep-2023,50
1019,Mercury,110,350000,15-Feb-2022,40
1005,Saturn,101,400000,01-Jun-2022,35
1025,Neptune,110,600000,01-Feb-2021,45


10. Explain your reasoning for your solution to #9 (use a Markdown cell), including who should get the award for Most Efficient Project Manager.

In [110]:
%%sql
SELECT firstName, lastName FROM Worker WHERE empId IN (
SELECT projMgrId FROM Project WHERE projNo == '1001')

 * sqlite:///worker_project.db
Done.


firstName,lastName
Tom,Smith


- First I considered the idea of Most Efficient to be the one with least budget/expectedDurationWeeks
- I then ordered by by the least and then limited the one with Most Efficient.
- The Most Efficient Project Manager in this case would be the one with the least budget/expectedDurationWeeks
- In this case Tom Smith should be awarded the most efficient Project Manager

11. Bonus!!  A Query that you can't easily answer with just a set of queries: Get a list of project numbers with name and start dates, of all the projects that have the same start date as another. 
*you don't have to answer this, but think about it!*


Count date occurance. The one with the count > 1, the return the project names and start dates

In [112]:
%%sql
SELECT projNo, projName
FROM
(SELECT startDate, Count(*) FROM Project  GROUP BY (startDate) LIMIT 1) as sdate
INNER Join Project on Project.startDate = sdate.startDate


 * sqlite:///worker_project.db
Done.


projNo,projName
1001,Jupiter
1025,Neptune


## Submission:

Commit and push your changes to this document by the deadline, and share a link to the repository in the Moodle assignment.  